# Metadata collection with ZnTrack

ZnTrack allows for the collection of some metadata.
One example is measuring the execution time of Nodes or even methods inside the Nodes easily.
This can be achieved by using the `@TimeIt` decorator which is shown in the following example.

In [1]:
!git init
!dvc init

Initialized empty Git repository in /tikhome/fzills/Examples/ZnTrack/meta_data/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [2]:
from zntrack import Node, dvc, config
from zntrack.metadata import TimeIt
from time import sleep

In [3]:
config.nb_name = "05_metadata.ipynb"

In [4]:
@Node(silent=True)
class SleepNode:
    @TimeIt
    def run(self):
        self.sleep_1s()
        self.sleep_2s()
    @TimeIt
    def sleep_1s(self):
        sleep(1)
    def sleep_2s(self):
        sleep(2)

2021-11-10 17:12:19,498 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 05_metadata.ipynb to script
[NbConvertApp] Writing 1321 bytes to 05_metadata.py


In [5]:
SleepNode()()

In [6]:
!dvc repro

Running stage 'SleepNode':                                            core>
> python3 -c "from src.SleepNode import SleepNode; SleepNode(load=True).run()"
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [7]:
!dvc metrics show

Path                          metadata.run:timeit    metadata.sleep_1s:timeit9m>
nodes/SleepNode/metrics.json  3.00369                1.00104


We can also time a sinlge function multiple times, using the following example:

In [8]:
@Node(silent=True)
class SleepNodeMulti:
    @TimeIt
    def run(self):
        self.sleep(1)
        self.sleep(2)
    @TimeIt
    def sleep(self, time):
        sleep(time)

2021-11-10 17:12:35,270 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.


[NbConvertApp] Converting notebook 05_metadata.ipynb to script
[NbConvertApp] Writing 1321 bytes to 05_metadata.py


In [9]:
SleepNodeMulti()()

In [10]:
!dvc repro

Stage 'SleepNode' didn't change, skipping                             core>
Running stage 'SleepNodeMulti':
> python3 -c "from src.SleepNodeMulti import SleepNodeMulti; SleepNodeMulti(load=True).run()"
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [11]:
!dvc metrics show

Path                               metadata.run:timeit    metadata.sleep:timeit    metadata.sleep_1:timeit    metadata.sleep_1s:timeit
nodes/SleepNode/metrics.json       3.00369                -                        -                          1.00104
nodes/SleepNodeMulti/metrics.json  3.00379                1.00129                  2.00205                    -


One can also access the metrics directly within Python. This is possible, because they are just another `zn.metrics` which is automatically added when using one of the given metadata decorators.

In [12]:
SleepNodeMulti(load=True).metadata

{'sleep:timeit': 1.0012879371643066,
 'sleep_1:timeit': 2.002049446105957,
 'run:timeit': 3.003790855407715}